In [1]:
import numpy as np
import pandas as pd
import snowflake.connector
import requests
import io
import os
import pywhatkit as kit
import time

In [2]:
#Conectar a snowflake con las credenciales
snowflake_connection = snowflake.connector.connect(user='esteban.correa@RAPPI.COM', 
                                   authenticator='externalbrowser', 
                                   account='hg51401', 
                                   warehouse="RP_PERSONALUSER_WH",
                                   database="FIVETRAN")

In [3]:
#Definir variable para la query de forecast
query_forecast = """
SELECT
    f.WAREHOUSE_ID,
    w.WAREHOUSE,
    f.PRODUCT_ID,
    w.PRODUCT_NAME,
    CASE 
        WHEN f.PRODUCT_ID = 3396 AND f.DATE = '2025-02-28' THEN f.FORECAST * 1.25
        ELSE f.FORECAST 
    END AS FORECAST,
    f.SALES_UNITS,
    f.DATE, 
    w.PROVIDER_NAME, 
    w.CAT_2
FROM RP_SILVER_DB_PROD.TURBO_CORE.GLOBAL_FORECAST_MAIN f
INNER JOIN (
    SELECT DISTINCT 
        WAREHOUSE,
        CITY,
        WAREHOUSE_ID,
        PROVIDER_NAME,
        RETAIL_ID,
        CAST(VIVO_ID AS NUMERIC) AS VIVO_ID,
        CAT_2,
        PRODUCT_NAME
    FROM RP_SILVER_DB_PROD.TURBO_SUPPLY.CO_WISHLIST_V1 
) w
ON f.WAREHOUSE_ID = CAST(w.WAREHOUSE_ID AS INT)
   AND f.PRODUCT_ID = w.RETAIL_ID
WHERE f.country = 'CO'
  AND f.DATE BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '14 DAYS'
  AND UPPER(w.CAT_2) = 'FRUTAS Y VERDURAS';
"""

In [4]:
#Ejecutar query de forecast en snowflake
df = pd.read_sql(query_forecast, snowflake_connection)

C:\Users\juane\AppData\Local\Temp\ipykernel_13736\4184599388.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_forecast, snowflake_connection)


In [5]:
for provider in df['PROVIDER_NAME'].unique():
    # Crear un archivo Excel para cada provider
    file_name = f"C:/Users/juane/Downloads/FORECAST_{provider}.xlsx"
    
    # Usar un ExcelWriter para agregar múltiples hojas en un archivo
    with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
        for store in df['WAREHOUSE'].unique():
            # Filtrar por provider y store
            temp_df = df[(df['PROVIDER_NAME'] == provider) & (df['WAREHOUSE'] == store)]
            
            if temp_df.empty:
                continue  # Saltar si no hay datos para esta combinación
            
            # Pivotear el DataFrame
            pivot_df = temp_df.pivot_table(
                index=["WAREHOUSE", "PRODUCT_ID", "PRODUCT_NAME"],  # Índices de la tabla pivoteada
                columns="DATE",                       # Las fechas se convierten en columnas
                values="FORECAST",                    # Valores a pivotear
                aggfunc="sum"                         # Sumar si hay duplicados
            ).reset_index()
            
            # Limpiar nombres de columnas
            pivot_df.columns.name = None  # Remueve el nombre de las columnas
            pivot_df = pivot_df.rename_axis(None, axis=1)  # Remueve índice nombrado
            
            # Escribir la hoja correspondiente a este store
            pivot_df.to_excel(writer, index=False, sheet_name=f"{store}")
            
    print(f"Archivo creado: {file_name}")


Archivo creado: C:/Users/juane/Downloads/FORECAST_Selvati.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Kompro2.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Experiencia Colombiana.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Kampos Fruver.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_TuPlaza.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Waruwa.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Verfrut.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Kompro.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_VERYFRUT.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_MandaleFruta.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Don Fruver.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_FRUVER FRESH.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_DISTRIFRUVER DEL EJE.xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Almacenes Exito S.A..xlsx
Archivo creado: C:/Users/juane/Downloads/FORECAST_Vive Agro